Вот ссылка на папку Orion_YOLO_Training https://drive.google.com/drive/folders/1YRZW8i9_gVn8ZcEtnJ_DQqLMTI2St5aV?usp=sharing. Добавьте ее себе на диск, прям перетащите в "Мой диск", это важно! Она будет выступать в роли хранилища результатов обучения моделей.

В ней уже есть несколько версий, например, yolov8x_obb2, которая обучалась приблизительно в течение 35 эпох на второй версии нашего датасета. После добавления этой папки вы сможете выполнять код с экспортом моделей из нее и сохранять туда результаты (это происходит автоматически во время выполнения функции train). Таким образом, у нас у всех будет возможность воспользоваться актуальной версией модели для экспериментов или дообучения


---

Сохранение текущего рабочего каталога в переменную HOME

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

Монтируем Google Drive в Colab, чтобы можно было загружать/сохранять файлы. Конкретно в коде это нужно для подгрузки предобученной модели или для сохранения весов во время обучения; что полезно, т.к. Colab ежедневно дает ограниченное кол-во ресурсов, и если не заметить как они закончатся, то можно потерять все данные, полученные во время сессии; а так будет возможность сохранять чекпоинты на диск

In [ ]:
from google.colab import drive
drive.mount(f'{HOME}/drive')

Проверка доступности GPU

In [ ]:
!nvidia-smi

Создание папки для датасета и загрузка Roboflow

In [ ]:
!mkdir -p {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow -q

from roboflow import Roboflow

rf = Roboflow(api_key="FoGVG6irzBU8JzOqY72L")
project = rf.workspace("orion-msydo").project("ship-detection-yr46o")
version = project.version(2)
dataset = version.download("yolov8-obb")


Установка и загрузка YOLO

In [ ]:
!pip install ultralytics -q

from ultralytics import YOLO

# Для обучения с нуля
#model = YOLO('yolov8x-obb.pt')

# Загрузка предобученной модели с диска
model= YOLO(f'{HOME}/drive/MyDrive/Orion_YOLO_Training/yolov8x_obb2/weights/last.pt')

# Экспорт из Roboflow; но к сожалению это не работает, так как вообще задеплоить obb модель в проект пока нельзя
#model_path = version.model.weights("yolov8-obb")
#model = YOLO(model_path)

Обновление конфигурационного файла для YOLO с obb



In [ ]:
import yaml

with open(f'{dataset.location}/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
data['train'] = '../train/images'
data['val'] = '../valid/images'
data['test'] = '../test/images'
if 'path' in data:
  del data['path']
with open(f'{dataset.location}/data.yaml', 'w') as f:
    yaml.dump(data, f, sort_keys=False)

---
Запуск обучения

In [ ]:
%cd {HOME}

results = model.train(
    data=f'{dataset.location}/data.yaml',
    epochs=50,
    imgsz=640,
    batch = 16,
    plots=True,
    project=f'{HOME}/drive/MyDrive/Orion_YOLO_Training', # Путь для сохранения на диск (каждую эпоху last.pl и best.pl)
    name="yolov8x_obb"
)

Вывод графика метрик после обучения (потери, точность и т.д.)

In [ ]:
from IPython.display import display
from PIL import Image

#display(Image.open(f'{HOME}/runs/obb/train/results.png'))

# С диска
display(Image.open(f'{HOME}/drive/MyDrive/Orion_YOLO_Training/yolov8x_obb2/results.png'))

Запуск предсказания на тестовых изображениях

In [ ]:
pre = model.predict(
    source=f'{dataset.location}/test/images',
    imgsz=640,
    save=True,
    name='predict',
    conf=0.25
)


Вывод изображений с предсказаниями




In [ ]:
import glob
from IPython.display import display
from PIL import Image

# Получаем список первых 5 предсказанных изображений
predicted_images = sorted(glob.glob(f'{HOME}/datasets/runs/obb/predict/*.jpg'))[:5]

for img_path in predicted_images:
    display(Image.open(img_path))
